# Biomedicina: Entrenamiento y clasificacion de datos EEG

In [1]:
import numpy as np
import pandas as pd
from pandas.tools.plotting import radviz
import matplotlib.pyplot as plt
from time import time

from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import RandomizedPCA
from sklearn.svm import SVC
#from sklearn.pipeline import Pipeline
#from sklearn.kernel_approximation import (RBFSampler, Nystroem)
import seaborn as sns; sns.set()

In [2]:
%matplotlib inline
plt.rcParams['figure.figsize'] = 16, 12

## Preprocesado de datos

In [3]:
s1 = '/home/cramos/Dropbox/BIO/train_subject1_raw01.asc'
s2 = '/home/cramos/Dropbox/BIO/train_subject1_raw02.asc'
s3 = '/home/cramos/Dropbox/BIO/train_subject1_raw03.asc'
t  = '/home/cramos/Dropbox/BIO/test_subject1_raw04.asc'
yr = '/home/cramos/Dropbox/BIO/res_subject1_raw.asc'
etiquetas = ['Fp1', 'AF3', 'F7', 'F3', 'FC1', 'FC5', 'T7', 'C3', 
             'CP1', 'CP5', 'P7', 'P3', 'Pz' , 'PO3', 'O1', 'Oz', 
             'O2' , 'PO4', 'P4', 'P8', 'CP6', 'CP2', 'C4', 'T8', 
             'FC6', 'FC2', 'F4', 'F8', 'AF4', 'Fp2', 'Fz', 'Cz', 
             'Clase']
sub1 = pd.read_fwf(s1, names=etiquetas)
sub2 = pd.read_fwf(s2, names=etiquetas)
sub3 = pd.read_fwf(s3, names=etiquetas)
tst  = pd.read_fwf(t, names=etiquetas[:32])
y_r  = pd.read_fwf(yr)


Tomamos los 3 datasets de entrenamiento y los concatenamos en un solo dataframe (subs).

In [4]:
frames = [sub1, sub2, sub3]
#subs = pd.concat(frames)
subs = sub1

In [ ]:
radviz(subs, 'Clase')

### Construcción de matrices de entrenamiento

In [5]:
X = subs.iloc[:,0:32].values
y = subs.iloc[:,32:].values

In [6]:
Xt = tst.iloc[:,0:32].values
y_real = np.matrix(y_r)

In [7]:
y = np.ravel(y)

In [ ]:
x_ind = 5
y_ind = 16

plt.scatter(X[:,x_ind], X[:,y_ind], c=y, cmap='RdYlBu')
plt.title('Dataset entrenamiento')
plt.ylabel(etiquetas[y_ind]),
plt.xlabel(etiquetas[x_ind])
plt.colorbar(label='Clases')

## Entrenamiento

In [ ]:
# Número de muestras
n_samples = X.shape[0]

# Número de características
n_features = X.shape[1]

# Número de clases
n_classes = np.unique(y).shape[0]


print("Tamaño total de dataset")
print("Muestras: %d" % n_samples)
print("Características: %d" % n_features)
print("Clases: %d" % n_classes)

###############################################################################
# Separación de muestras a set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)


###############################################################################
# Cálculo de PCA en dataset (se trata como un dataset sin etiquetas):
# extracción de características no supervisada / reducción dimensional

n_components = 32

print("* Extrayendo las %d características superiores de las %d muestras"
      % (n_components, X_train.shape[0]))
t0 = time()
pca = RandomizedPCA(n_components=n_components, whiten=True).fit(X_train)
print("finalizado en %0.3fs" % (time() - t0))


print("* Proyectando los datos de entrada en las características seleccionadas")
t0 = time()
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print("finalizado en %0.3fs" % (time() - t0))


###############################################################################
# Entrenamiento de clasificador SVM

print("* Ajustando el clasificador SVM con el conjunto de entrenamiento")
t0 = time()
param_grid = {'C': [1e3],
              'gamma': [2], }
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)
clf = clf.fit(X_train_pca, y_train)
print("finalizado en %0.3fs" % (time() - t0))
print("Mejor estimador encontrado mediante grid search:")
print(clf.best_estimator_)
clf = clf.best_estimator_
###############################################################################
# Evaluación cuantitativa de la calidad del modelo en el set de testeo

print("* Predicción de las clases del test set")
t0 = time()
y_pred = clf.predict(X_test_pca)
print("finalizado en %0.3fs" % (time() - t0))

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred, labels=range(n_classes)))


Tamaño total de dataset
Muestras: 122368
Características: 32
Clases: 3
* Extrayendo las 32 características superiores de las 91776 muestras
finalizado en 1.836s
* Proyectando los datos de entrada en las características seleccionadas
finalizado en 0.126s

## Clasificación

In [9]:
Yt = clf.predict(Xt)

ValueError: X.shape[1] = 32 should be equal to 5, the number of features at training time

Después de predecir volvemos a construir el dataframe con los resultados:

In [ ]:
Ytm=np.matrix(Yt)

In [ ]:
subt = np.concatenate((Xt,Ytm.T), axis=1)

In [ ]:
subt = pd.DataFrame(subt, columns=etiquetas)

## Ploteo de resultados

In [ ]:
radviz(subt, 'Clase')

In [ ]:
Yt_r = np.array(Yt).T[:][:470]

In [ ]:
x_ind = 5
y_ind = 16

plt.subplot(2, 1, 1)
plt.scatter(Xtt[:,x_ind], Xtt[:,y_ind], c=np.array(Yt_r), cmap='RdYlBu')
plt.title('Clasificación algoritmo')
plt.ylabel(etiquetas[y_ind]),
plt.xlabel(etiquetas[x_ind])
plt.colorbar(label='Clases')

plt.subplot(2, 1, 2)
plt.scatter(Xtt[:,x_ind], Xtt[:,y_ind], c=np.array(y_real), cmap='RdYlBu')
plt.title('Clasificación real')
plt.ylabel(etiquetas[y_ind]),
plt.xlabel(etiquetas[x_ind])
plt.colorbar(label='Clases',)

plt.show()